In [119]:
import warnings
import random
import time

from vizdoom import *

import numpy as np
from skimage import transform, util, color
from collections import deque
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [94]:
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
que_length = 100
total_episodes = 50000        # Total episode
learning_rate = 0.7           # Learning rate
max_steps = 999                # Max steps per episode
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [89]:
def createEnvironment():
    game = DoomGame()
    game.load_config("scenarios/basic.cfg")
    game.init()
    
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    actions = [left, right, shoot]
    
    return game, actions

In [106]:
def processImg(img):
    img = color.rgb2gray(img)
    img = img[60:, :]
    img = img / 255
    img = transform.resize(img, (200, 200))
    return img

In [ ]:
stack_size = 4
stacked_frames = deque([np.zeros((200,200), dtype=np.int) for i in range(stack_size)], maxlen=4)
def stackFrames(stacked_frames, stateim, is_new_episode):
    frame = processImg(stateim)
    if is_new_episode:
        stacked_frames = deque([np.zeros((200,200), dtype=np.int) for i in range(stack_size)], maxlen=4)
        for _ in range(stack_size):
            stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
    else:
        stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
        
    return stacked_state, stacked_frames

In [ ]:
mem = deque([], maxlen=que_length)
def experienceReplay():

In [ ]:
class DQN(nn.Module):
    def __init__(self, img_size):
        self.img_size = img_size
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=8, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 3)
        
    def forward(self, x):
        x = F.elu(self.bn1(self.conv1(x)))
        x = F.elu(self.bn2(self.conv2(x)))
        x = F.elu(self.bn3(self.conv3(x)))
        
        x = x.view(-1, 128)
        x = self.fc1(x)
        x = F.elu(self.fc2(x))
        return x

In [ ]:
def run():
    game, actions = createEnvironment()
    mem = deque([], maxlen=que_length)
    for episode in range(total_episodes):
        state = game.get_state()
        
        for step in range(max_steps):
            state_im = rescaleAndGray(state.screen_buffer)
            
            exp_exp_tradeoff = random.uniform(0,1)
            if exp_exp_tradeoff > epsilon:
                action = nnet.predict(state_im)
            else:
                action = random.choice(actions)
                
            reward = game.make_action(action)
            new_s = game.get_state()
            new_im = rescaleAndGray(new_s.screen_buffer)
            
            mem.append((state_im, action, reward, new_im))
            state = new_s
            
            Q_target = mem[mem.index(state_im)][2] + gamma*
        
        episode += 1
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

In [32]:
state = game.get_state()

img = state.screen_buffer

img.shape

img = np.moveaxis(img, 0, 2)

img.shape

plt.imshow(img)
plt.show()